In [14]:
# importing function from a different ipny 
import pandas as pd
import collections
from keras.initializers import Constant
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import operator
from itertools import product
import sys
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from functools import reduce
import spacy
from spacy.tokenizer import Tokenizer
nlp = spacy.load("en_core_web_sm")
tokenizer = Tokenizer(nlp.vocab)
pd.options.display.max_columns = 1000
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Activation, Input, BatchNormalization, MaxPooling1D, Bidirectional,LSTM
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPool1D, Flatten , Embedding, GlobalMaxPool1D
from keras.models import Model
from keras.wrappers.scikit_learn import KerasClassifier
#https://www.amazon.com/Neural-Networks-Deep-Learning-Textbook/dp/3319944622/ref=cm_cr_arp_d_product_top?ie=UTF8
#https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6

In [15]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_NUM_WORDS = 20000

In [16]:
keras_data = pd.read_csv('subject_spam.csv', index_col=0, encoding='utf8', engine='python')
keras_data.fillna("_na_", inplace = True)
keras_data = keras_data.drop(keras_data[keras_data.isspam == "_na_"].index)
spammer = {'spam ': 1,'ham ': 0} 
keras_data.isspam = [spammer[item] for item in keras_data.isspam] 
spam_index = keras_data[keras_data.isspam ==1].index
ham_index = keras_data[keras_data.isspam == 0].index
new_index = np.concatenate((spam_index[:10000], ham_index[:9997]), axis=0)
keras_data_new = keras_data.iloc[new_index]
labels = keras_data_new.isspam
keras_data_new.head()

,Subject,isspam
0,"Confidential :Soma:, Ci@lis, :P:ntermin",1
1,¢Â ÇªÁüÇÑ ¼³³¯ ÀÌº¥Æ® ÀÀ¸ðÇØ¼­ ºÎ¸ð´Ô²² ¼±¹°ÇÏ...,1
2,_na_,1
3,StOck 0ppurtunities - their sh0Oting,1
6,All your prescr[iption needs right here,1


In [17]:
VALIDATION_SPLIT = 0.2
MAX_SEQUENCE_LENGTH = 1000
EMBEDDING_DIM = 100

In [18]:
embeddings_index = {}
with open(os.path.join('glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

# second, prepare text samples and their labels
print('Processing text dataset')

Found 400000 word vectors.
Processing text dataset


In [19]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(keras_data_new.Subject)
sequences = tokenizer.texts_to_sequences(keras_data_new.Subject)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

#labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]



Found 15425 unique tokens.
Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997,)


In [20]:
print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

print('Training model.')



Preparing embedding matrix.
Training model.


# Embedding Model

In [21]:
keras_data = pd.read_csv('subject_spam.csv', index_col=0, encoding='utf8', engine='python')
keras_data.fillna("_na_", inplace = True)
keras_data = keras_data.drop(keras_data[keras_data.isspam == "_na_"].index)
spammer = {'spam ': 1,'ham ': 0} 
keras_data.isspam = [spammer[item] for item in keras_data.isspam] 
spam_index = keras_data[keras_data.isspam ==1].index
ham_index = keras_data[keras_data.isspam == 0].index
new_index = np.concatenate((spam_index[10000:19000], ham_index[9997:19000]), axis=0)
keras_data_test_set = keras_data.iloc[new_index]
labels_test_set = keras_data_test_set.isspam
keras_data_test_set.head()

,Subject,isspam
19223,RE: Daily Summary of Risk Data,0
19224,Softwares CDS all software under 15$ and 99$!,1
19226,Hotel Room Bargains at up to 70% off! Save in...,1
19227,RE: Greeley Gas Company,0
19228,"Any Software just in under 15-99$, Xp-adobe etc",1


In [22]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(keras_data_new.Subject)
sequences = tokenizer.texts_to_sequences(keras_data_test_set.Subject)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

#labels_test_set = to_categorical(np.asarray(labels_test_set))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
#labels_test_set = labels_test_set[indices]
x_test = data

Found 15425 unique tokens.
Shape of data tensor: (17088, 1000)
Shape of label tensor: (19997,)


In [23]:
embedding = pd.read_csv('embeddings_total.csv')
result_sort = embedding.sort_values(['val_loss'])

In [24]:
#result = pd.concat([embedding1, embedding], axis=0)
#result_sort = result.sort_values(['val_loss'])
result_sort.head()

,dl,fl1,fl2,fl3,kl,layer,optimizer,train_acc,train_loss,val_acc,val_loss,trainable_params
1989,16,128,16,32,5,1,Adam,0.921803,0.181132,0.901725,0.230709,66226
2026,64,128,16,64,5,2,Adam,0.917302,0.191708,0.902476,0.231941,75602
1976,32,128,16,16,5,3,Nadam,0.918865,0.190353,0.900975,0.233088,76290
1813,64,64,64,32,5,2,Nadam,0.915677,0.195752,0.895224,0.233315,56898
1987,128,128,16,16,5,2,Nadam,0.918740,0.188945,0.899725,0.234236,76818


In [25]:
def embeddings(fl1=32, fl2=32, fl3=64, dl=16, optimizer= 'RMSprop', kl = 5, layer =1 ):
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    if (layer == 1):
        x = Conv1D(filters = fl1, kernel_size = kl, activation='relu')(embedded_sequences)
        x = MaxPooling1D(pool_size = kl)(x)
    elif (layer == 2):
        x = Conv1D(filters = fl1, kernel_size = kl, activation='relu')(embedded_sequences)
        x = MaxPooling1D(pool_size = kl)(x)
        x = Conv1D(filters = fl2, kernel_size = kl, activation='relu')(x)
        x = MaxPooling1D(pool_size = kl)(x)
        
    else:
        x = Conv1D(filters = fl1, kernel_size = kl, activation='relu')(embedded_sequences)
        x = MaxPooling1D(pool_size = kl)(x)
        x = Conv1D(filters = fl2, kernel_size = kl, activation='relu')(x)
        x = MaxPooling1D(pool_size = kl)(x)
        x = Conv1D(filters = fl3, kernel_size = kl, activation='relu')(x)
    x = GlobalMaxPooling1D()(x)
    x = Dense(units = dl, activation='relu')(x)
    preds = Dense(1, activation='tanh')(x)
    model = Model(sequence_input, preds)
    model.compile(loss= 'binary_crossentropy',
              optimizer= optimizer,
              metrics=['acc'])
   
    return model

## 1st model

In [26]:
kwargs = dict(fl1=128, fl2= 0, fl3=0, kl=5, dl=16, optimizer= ''.join('Adam'), layer=1)
model = embeddings(**kwargs)
model.fit(x_train, y_train, batch_size= 16, epochs=2, validation_data=(x_val, y_val))
model_json = model.to_json()
with open("model_embeddings_1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_embeddings_1.h5")
print("Saved model to disk")
model1_test_y = model.predict(x_test, batch_size=16, verbose=1)

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 372s 23ms/step - loss: 0.5869 - acc: 0.6908 - val_loss: 0.5413 - val_acc: 0.6989
Epoch 2/2
15998/15998 [==============================] - 325s 20ms/step - loss: 0.4385 - acc: 0.7536 - val_loss: 0.5013 - val_acc: 0.7219
Saved model to disk
17088/17088 [==============================] - 131s 8ms/step


## 2nd model

In [27]:
kwargs = dict(fl1=128, fl2= 16, fl3=0, kl=5, dl=64, optimizer= ''.join('Adam'), layer=2)
model = embeddings(**kwargs)
model.fit(x_train, y_train, batch_size= 16, epochs=2, validation_data=(x_val, y_val))
model_json = model.to_json()
with open("model_embeddings_2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_embeddings_2.h5")
print("Saved model to disk")
model2_test_y = model.predict(x_test, batch_size=16, verbose=1)

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 314s 20ms/step - loss: 0.5111 - acc: 0.7665 - val_loss: 0.4755 - val_acc: 0.7404
Epoch 2/2
15998/15998 [==============================] - 311s 19ms/step - loss: 0.4056 - acc: 0.7147 - val_loss: 0.4097 - val_acc: 0.7319
Saved model to disk
17088/17088 [==============================] - 134s 8ms/step


## 3rd model

In [28]:
kwargs = dict(fl1=128, fl2= 16, fl3=16, kl=5, dl=32, optimizer= ''.join('Nadam'), layer=3)
model = embeddings(**kwargs)
model.fit(x_train, y_train, batch_size= 16, epochs=2, validation_data=(x_val, y_val))
model_json = model.to_json()
with open("model_embeddings_3.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_embeddings_3.h5")
print("Saved model to disk")
model3_test_y = model.predict(x_test, batch_size=16, verbose=1)

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 319s 20ms/step - loss: 0.4482 - acc: 0.8016 - val_loss: 0.3709 - val_acc: 0.8652
Epoch 2/2
15998/15998 [==============================] - 312s 20ms/step - loss: 0.3344 - acc: 0.8572 - val_loss: 0.3456 - val_acc: 0.8525
Saved model to disk
17088/17088 [==============================] - 134s 8ms/step


### 4th Model

In [29]:
kwargs = dict(fl1=64, fl2= 64, fl3=0, kl=5, dl=64, optimizer= ''.join('Nadam'), layer=2)
model = embeddings(**kwargs)
model.fit(x_train, y_train, batch_size= 16, epochs=2, validation_data=(x_val, y_val))
model_json = model.to_json()
with open("model_embeddings_4.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_embeddings_4.h5")
print("Saved model to disk")
model4_test_y = model.predict(x_test, batch_size=16, verbose=1)

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 233s 15ms/step - loss: 0.5724 - acc: 0.6875 - val_loss: 0.3983 - val_acc: 0.7444
Epoch 2/2
15998/15998 [==============================] - 229s 14ms/step - loss: 0.4042 - acc: 0.7015 - val_loss: 0.4025 - val_acc: 0.7619
Saved model to disk
17088/17088 [==============================] - 98s 6ms/step


### 5th Model

In [30]:
kwargs = dict(fl1=128, fl2= 16, fl3=0, kl=5, dl=128, optimizer= ''.join('Nadam'), layer=2)
model = embeddings(**kwargs)
model.fit(x_train, y_train, batch_size= 16, epochs=2, validation_data=(x_val, y_val))
model_json = model.to_json()
with open("model_embeddings_5.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_embeddings_5.h5")
print("Saved model to disk")
model5_test_y = model.predict(x_test, batch_size=16, verbose=1)

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 324s 20ms/step - loss: 0.5277 - acc: 0.7188 - val_loss: 0.4324 - val_acc: 0.7649
Epoch 2/2
15998/15998 [==============================] - 325s 20ms/step - loss: 0.4147 - acc: 0.7373 - val_loss: 0.3594 - val_acc: 0.8077
Saved model to disk
17088/17088 [==============================] - 139s 8ms/step


In [32]:
def F1_score(pred_test_y, actuals):

    predictions =[]
    true_pos = 0
    true_neg = 0
    false_pos = 0
    false_neg = 0

    for i in range(len(pred_test_y)):
        if (pred_test_y[i]>=0.5):
            predictions.append(1)
        else:
            predictions.append(0)
    
    for i in range (len(pred_test_y)):
        if ((predictions[i]==1) & (actuals[i]==1)):
            true_pos = true_pos+1
        elif((predictions[i]==0) & (actuals[i]==0)):
            true_neg = true_neg+1
        elif((predictions[i]==1) & (actuals[i]==0)):
            false_pos = false_pos +1
        elif((predictions[i]==0) & (actuals[i]==1)):
            false_neg = false_neg+1
    prec=true_pos/(true_pos+false_pos)
    recall = true_pos/(true_pos+false_neg)
    accur=(true_pos+true_neg)/(true_pos+false_pos+ true_neg+ false_neg)
    F1=2*(prec*recall/(prec+recall))
    FPR = false_pos/(false_pos+true_neg)
    return (true_pos, false_pos, true_neg, false_neg, prec,FPR)

In [37]:
print((F1_score(model1_test_y, labels_test_set)))
print((F1_score(model2_test_y, labels_test_set)))
print((F1_score(model3_test_y, labels_test_set)))
print((F1_score(model4_test_y, labels_test_set)))
print((F1_score(model5_test_y, labels_test_set)))

(6603, 5894, 2169, 2422, 0.5283668080339281, 0.7309934267642316)
(5133, 4572, 3491, 3892, 0.528902627511592, 0.567034602505271)
(4899, 4377, 3686, 4126, 0.5281371280724451, 0.5428500558104924)
(5539, 4947, 3116, 3486, 0.5282281136753767, 0.613543346149076)
(5287, 4698, 3365, 3738, 0.529494241362043, 0.5826615403695895)
